In [1]:
import pandas as pd
import re
from nltk.corpus import stopwords
import pickle
import numpy as np
from string import punctuation
import time
import spacy
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [2]:
# Import regulatory sections with matched noun chunks
df_regSentsExpand=pd.read_pickle('/home/ec2-user/SageMaker/New Uncertainty/Reg Relevance/RegSentsExpand_NounChunks3.pkl')
print(df_regSentsExpand.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 822737 entries, 0 to 822736
Data columns (total 6 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   ID                           822737 non-null  object
 1   RegSentsExpand               822737 non-null  object
 2   NounChunksMatch              822737 non-null  int64 
 3   NounChunkMatchWords          822737 non-null  object
 4   NounChunkMatchWordsFiltered  822737 non-null  object
 5   NounChunkMatchFiltered       822737 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 37.7+ MB
None


In [3]:
# Refine to reg relevant articles
df=df_regSentsExpand[df_regSentsExpand['NounChunkMatchFiltered']>0].reset_index(drop=True)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502909 entries, 0 to 502908
Data columns (total 6 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   ID                           502909 non-null  object
 1   RegSentsExpand               502909 non-null  object
 2   NounChunksMatch              502909 non-null  int64 
 3   NounChunkMatchWords          502909 non-null  object
 4   NounChunkMatchWordsFiltered  502909 non-null  object
 5   NounChunkMatchFiltered       502909 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 23.0+ MB
None


## 1. Match uncertainty words using LM dictionary

In [4]:
# Negation words
negate = ["aint", "arent", "cannot", "cant", "couldnt", "darent", "didnt", "doesnt", "ain't", "aren't", "can't",
          "couldn't", "daren't", "didn't", "doesn't", "dont", "hadnt", "hasnt", "havent", "isnt", "mightnt", "mustnt",
          "neither", "don't", "hadn't", "hasn't", "haven't", "isn't", "mightn't", "mustn't", "neednt", "needn't",
          "never", "none", "nope", "nor", "not", "nothing", "nowhere", "oughtnt", "shant", "shouldnt", "wasnt",
          "werent", "oughtn't", "shan't", "shouldn't", "wasn't", "weren't", "without", "wont", "wouldnt", "won't",
          "wouldn't", "rarely", "seldom", "despite", "no", "nobody"]

In [5]:
# Function to negate
def negated(word):
    # Determine if preceding word is a negation word
    if word.lower() in negate:
        return True
    else:
        return False

In [6]:
# Function to tokenize
def tokenizer(text):
    doc=nlp(text)
    tokens=[token.text.lower() for token in doc if not token.is_punct | token.is_space]
    return tokens

In [7]:
# Function to lemmatize
def lemmatizer(text):
    doc=nlp(text)
    lemmas=[token.lemma_ for token in doc if not token.is_punct | token.is_space]
    return lemmas

In [8]:
# LM dictionary
LMlist=pd.read_csv('/home/ec2-user/SageMaker/New Uncertainty/Sentiment Analysis/LoughranMcDonald_SentimentList.csv')
print(LMlist.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2355 entries, 0 to 2354
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Negative     2355 non-null   object
 1   Positive     354 non-null    object
 2   Uncertainty  297 non-null    object
dtypes: object(3)
memory usage: 55.3+ KB
None


In [9]:
# LM uncertainty dictionary
LMuncertain=LMlist[LMlist['Uncertainty'].notnull()]['Uncertainty'].tolist()
uncertaindict={'Uncertainty': [w.lower() for w in LMuncertain]}
#print(uncertaindict, len(LMuncertain))

In [10]:
# Lemmatize LM uncertainty dictionary
uncertainset=set()
for w in uncertaindict['Uncertainty']:
    v=''.join(lemmatizer(w))
    uncertainset.add(v)
uncertainlist_lemmatized=list(uncertainset)
print(len(uncertainlist_lemmatized))

213


In [11]:
print(uncertainlist_lemmatized[0:20])

['hide', 'nonassessable', 'alteration', 'clarification', 'imprecision', 'imprecise', 'vaguely', 'differ', 'indeterminate', 'unforseen', 'cautious', 'apparently', 'approximated', 'recalculated', 'arbitrary', 'indefinite', 'confusion', 'crossroad', 'hinge', 'possibility']


In [12]:
# Function to count uncertainty terms
def uncertainty_count(keywords_list, article):

    uncertain_count = 0
    uncertain_words = []
 
    input_words=lemmatizer(article)
    word_count = len(input_words)
    
    for i in range(0, word_count):
        if input_words[i] in keywords_list:
            uncertain_count += 1
            uncertain_words.append(input_words[i])
    
    results = [uncertain_count, uncertain_words]
 
    return results

In [13]:
# Run LM uncertainty through all expanded reg sentences
UncertaintyCount=[]
UncertaintyWords=[]
for text in df['RegSentsExpand']:
    results=uncertainty_count(uncertainlist_lemmatized, text)
    UncertaintyCount.append(results[0])
    UncertaintyWords.append(results[1])
print(len(UncertaintyCount))

502909


In [14]:
df['UncertaintyCount']=UncertaintyCount
df['UncertaintyWords']=UncertaintyWords
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502909 entries, 0 to 502908
Data columns (total 8 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   ID                           502909 non-null  object
 1   RegSentsExpand               502909 non-null  object
 2   NounChunksMatch              502909 non-null  int64 
 3   NounChunkMatchWords          502909 non-null  object
 4   NounChunkMatchWordsFiltered  502909 non-null  object
 5   NounChunkMatchFiltered       502909 non-null  int64 
 6   UncertaintyCount             502909 non-null  int64 
 7   UncertaintyWords             502909 non-null  object
dtypes: int64(3), object(5)
memory usage: 30.7+ MB
None


In [15]:
print(df[df['UncertaintyCount']!=0]['ID'].nunique())

273029


In [18]:
print(df[['ID','UncertaintyCount','UncertaintyWords']].head(10))

          ID  UncertaintyCount   UncertaintyWords
0  294326637                 0                 []
1  294308147                 0                 []
2  294323196                 1            [could]
3  294262284                 2  [possible, could]
4  294234319                 0                 []
5  294323101                 1         [exposure]
6  294205735                 1           [revise]
7  294261564                 0                 []
8  294337453                 1            [pende]
9  294281403                 2     [appear, seem]


In [19]:
df[['ID','UncertaintyCount','UncertaintyWords']].to_csv('/home/ec2-user/SageMaker/New Uncertainty/Sentiment Analysis/LMuncertainty.csv',index=False)

## 2. Match sentiment words

### 2.1 LM dictionary

In [20]:
# Function to count sentiment terms
def sentiment_count(dict, article):
    """
    Count positive and negative words with negation check. Account for simple negation only for positive words.
    Simple negation is taken to be observations of one of negate words occurring within three words
    preceding a positive words.
    """
    pos_count = 0
    neg_count = 0
 
    pos_words = []
    neg_words = []
 
    input_words=lemmatizer(article)
 
    word_count = len(input_words)
 
    for i in range(0, word_count):
        if input_words[i] in dict['Negative']:
            if i >= 3:
                if negated(input_words[i - 1]) or negated(input_words[i - 2]) or negated(input_words[i - 3]):
                    pos_count += 1
                    pos_words.append(input_words[i] + ' (with negation)')
                else:
                    neg_count += 1
                    neg_words.append(input_words[i])
            elif i == 2:
                if negated(input_words[i - 1]) or negated(input_words[i - 2]):
                    pos_count += 1
                    pos_words.append(input_words[i] + ' (with negation)')
                else:
                    neg_count += 1
                    neg_words.append(input_words[i])
            elif i == 1:
                if negated(input_words[i - 1]):
                    pos_count += 1
                    pos_words.append(input_words[i] + ' (with negation)')
                else:
                    neg_count += 1
                    neg_words.append(input_words[i])
            elif i == 0:
                neg_count += 1
                neg_words.append(input_words[i])
            
        if input_words[i] in dict['Positive']:
            if i >= 3:
                if negated(input_words[i - 1]) or negated(input_words[i - 2]) or negated(input_words[i - 3]):
                    neg_count += 1
                    neg_words.append(input_words[i] + ' (with negation)')
                else:
                    pos_count += 1
                    pos_words.append(input_words[i])
            elif i == 2:
                if negated(input_words[i - 1]) or negated(input_words[i - 2]):
                    neg_count += 1
                    neg_words.append(input_words[i] + ' (with negation)')
                else:
                    pos_count += 1
                    pos_words.append(input_words[i])
            elif i == 1:
                if negated(input_words[i - 1]):
                    neg_count += 1
                    neg_words.append(input_words[i] + ' (with negation)')
                else:
                    pos_count += 1
                    pos_words.append(input_words[i])
            elif i == 0:
                pos_count += 1
                pos_words.append(input_words[i])
    '''
    print('The results with negation check:', end='\n\n')
    print('The # of positive words:', pos_count)
    print('The # of negative words:', neg_count)
    print('The list of found positive words:', pos_words)
    print('The list of found negative words:', neg_words)
    print('\n', end='')
    '''
    
    results = [word_count, pos_count, neg_count, pos_words, neg_words]
 
    return results

In [21]:
# LM sentiment dictionary
LMposWords=LMlist[LMlist['Positive'].notnull()]['Positive'].tolist()
LMnegWords=LMlist[LMlist['Negative'].notnull()]['Negative'].tolist()
print(len(LMnegWords),len(LMposWords))
print(LMnegWords[0:20],LMposWords[0:20])

2355 354
['ABANDON', 'ABANDONED', 'ABANDONING', 'ABANDONMENT', 'ABANDONMENTS', 'ABANDONS', 'ABDICATED', 'ABDICATES', 'ABDICATING', 'ABDICATION', 'ABDICATIONS', 'ABERRANT', 'ABERRATION', 'ABERRATIONAL', 'ABERRATIONS', 'ABETTING', 'ABNORMAL', 'ABNORMALITIES', 'ABNORMALITY', 'ABNORMALLY'] ['ABLE', 'ABUNDANCE', 'ABUNDANT', 'ACCLAIMED', 'ACCOMPLISH', 'ACCOMPLISHED', 'ACCOMPLISHES', 'ACCOMPLISHING', 'ACCOMPLISHMENT', 'ACCOMPLISHMENTS', 'ACHIEVE', 'ACHIEVED', 'ACHIEVEMENT', 'ACHIEVEMENTS', 'ACHIEVES', 'ACHIEVING', 'ADEQUATELY', 'ADVANCEMENT', 'ADVANCEMENTS', 'ADVANCES']


In [22]:
# Lemmatize LM sentiment dictionary
LMnegset=set()
for w in LMnegWords:
    v=''.join(lemmatizer(w.lower()))
    LMnegset.add(v)
print(len(LMnegset))

LMposset=set()
for w in LMposWords:
    v=''.join(lemmatizer(w.lower()))
    LMposset.add(v)
print(len(LMposset))

LMdict={'Negative': list(LMnegset), 'Positive': list(LMposset)}

1463
245


In [23]:
print(LMdict['Positive'][0:20])
print(LMdict['Negative'][0:20])

['charitable', 'dream', 'satisfie', 'bolstering', 'ideal', 'excited', 'smoothly', 'creative', 'bolster', 'boom', 'revolutionizes', 'complimenting', 'able', 'favorable', 'preeminence', 'diligently', 'easy', 'honorable', 'accomplishing', 'exclusively']
['incarcerate', 'obsolescence', 'incapacitate', 'miss', 'renounced', 'uneconomic', 'unfunded', 'unacceptably', 'deceitful', 'litigant', 'retaliated', 'discourage', 'stoppage', 'unfavorability', 'alienation', 'repudiation', 'expose', 'unsaleable', 'nonproductive', 'shortfall']


In [24]:
# Run LM sentiment through all expanded reg sentences
LMpositiveCount=[]
LMnegativeCount=[]
LMpositiveWords=[]
LMnegativeWords=[]
for text in df['RegSentsExpand']:
    results=sentiment_count(LMdict, text)
    LMpositiveCount.append(results[1])
    LMnegativeCount.append(results[2])
    LMpositiveWords.append(results[3])
    LMnegativeWords.append(results[4])

In [26]:
df['LMposCount']=LMpositiveCount
df['LMnegCount']=LMnegativeCount
df['LMposWords']=LMpositiveWords
df['LMnegWords']=LMnegativeWords

In [27]:
df[['ID','LMposCount','LMnegCount','LMposWords','LMnegWords']].to_csv('/home/ec2-user/SageMaker/New Uncertainty/Sentiment Analysis/LMsentiments.csv',index=False)

### 2.2 GI dictionary

In [28]:
# Harvard GI sentiment dictionary
with open("/home/ec2-user/SageMaker/New Uncertainty/Sentiment Analysis/GIposWords.txt", "rb") as fp:   # Unpickling
    GIposWords = pickle.load(fp)
with open("/home/ec2-user/SageMaker/New Uncertainty/Sentiment Analysis/GInegWords.txt", "rb") as fp:   # Unpickling
    GInegWords = pickle.load(fp)

In [29]:
print(len(GIposWords),GIposWords[0:20])
print(len(GInegWords),GInegWords[0:20])

1637 ['ABIDE', 'ABILITY', 'ABLE', 'ABOUND', 'ABSOLVE', 'ABSORBENT', 'ABSORPTION', 'ABUNDANCE', 'ABUNDANT', 'ACCEDE', 'ACCENTUATE', 'ACCEPT', 'ACCEPTABLE', 'ACCEPTANCE', 'ACCESSIBLE', 'ACCESSION', 'ACCLAIM', 'ACCLAMATION', 'ACCOLADE', 'ACCOMMODATE']
2005 ['ABANDON', 'ABANDONMENT', 'ABATE', 'ABDICATE', 'ABHOR', 'ABJECT', 'ABNORMAL', 'ABOLISH', 'ABOMINABLE', 'ABRASIVE', 'ABRUPT', 'ABSCOND', 'ABSENCE', 'ABSENT', 'ABSENT-MINDED', 'ABSENTEE', 'ABSURD', 'ABSURDITY', 'ABUSE', 'ABYSS']


In [30]:
# # Lemmatize GI dictionary
# GInegset=set()
# for w in GInegWords:
#     v=''.join(lemmatizer(w.lower()))
#     GInegset.add(v)
# GInegset.remove('-PRON-')
# print(len(GInegset))

# GIposset=set()
# for w in GIposWords:
#     v=''.join(lemmatizer(w.lower()))
#     GIposset.add(v)
# print(len(GIposset))

# GIdict={'Negative': list(GInegset), 'Positive': list(GIposset)}

In [31]:
# Non-lemmetized version of GI dictionary
GIdict2={'Negative': [w.lower() for w in GInegWords], 'Positive': [w.lower() for w in GIposWords]}
print('Positive:',GIdict2['Positive'][0:20])
print('Negative:',GIdict2['Negative'][0:20])

Positive: ['abide', 'ability', 'able', 'abound', 'absolve', 'absorbent', 'absorption', 'abundance', 'abundant', 'accede', 'accentuate', 'accept', 'acceptable', 'acceptance', 'accessible', 'accession', 'acclaim', 'acclamation', 'accolade', 'accommodate']
Negative: ['abandon', 'abandonment', 'abate', 'abdicate', 'abhor', 'abject', 'abnormal', 'abolish', 'abominable', 'abrasive', 'abrupt', 'abscond', 'absence', 'absent', 'absent-minded', 'absentee', 'absurd', 'absurdity', 'abuse', 'abyss']


In [ ]:
# Run GI sentiment through all expanded reg sentences using non-lemmatized GI dictionary (performs better than lemmatized GI)
totalWordCount=[]
GIpositiveCount=[]
GInegativeCount=[]
GIpositiveWords=[]
GInegativeWords=[]
for text in df['RegSentsExpand']:
    results=sentiment_count(GIdict2, text)
    totalWordCount.append(results[0])
    GIpositiveCount.append(results[1])
    GInegativeCount.append(results[2])
    GIpositiveWords.append(results[3])
    GInegativeWords.append(results[4])

In [ ]:
df['TotalWordCount']=totalWordCount
df['GIposCount']=GIpositiveCount
df['GInegCount']=GInegativeCount
df['GIposWords']=GIpositiveWords
df['GInegWords']=GInegativeWords

In [ ]:
df[['ID','TotalWordCount','GIposCount','GInegCount','GIposWords','GInegWords']].to_csv('/home/ec2-user/SageMaker/New Uncertainty/Sentiment Analysis/GIsentiments.csv',index=False)

### 2.3 LSD dictionary

In [ ]:
# Lexicoder Sentiment Dictionary (LSD)
LSDlist=pd.read_csv('/home/ec2-user/SageMaker/New Uncertainty/Sentiment Analysis/LSDsentimentWords_wStar.csv')
print(LSDlist.info())

In [ ]:
LSDneg=LSDlist[LSDlist['LSDnegative'].notnull()]['LSDnegative'].tolist()
LSDpos=LSDlist[LSDlist['LSDpositive'].notnull()]['LSDpositive'].tolist()
LSDdict={'Negative': [w.lower() for w in LSDneg], 'Positive': [w.lower() for w in LSDpos]}

In [ ]:
# Seperate terms with & without stars in LSD dictionary
pos_star=[]
pos_nostar=[]
for m in LSDdict['Positive']:
    if "*" in m:
        m=m.replace('*','')
        pos_star.append(m)
    else:
        pos_nostar.append(m)
print(len(pos_star), len(pos_nostar))

neg_star=[]
neg_nostar=[]
for m in LSDdict['Negative']:
    if "*" in m:
        m=m.replace('*','')
        neg_star.append(m)
    else:
        neg_nostar.append(m)
print(len(neg_star), len(neg_nostar))

In [ ]:
# Compile re patterns for terms with & without stars
pattern_pos_nostar=re.compile(r'\b(?:%s)\b' % '|'.join(pos_nostar))
pattern_pos_star=re.compile(r'\b(?:%s)[a-zA-Z]*\b' % '|'.join(pos_star))
pattern_neg_nostar=re.compile(r'\b(?:%s)\b' % '|'.join(neg_nostar))
pattern_neg_star=re.compile(r'\b(?:%s)[a-zA-Z]*\b' % '|'.join(neg_star))

In [ ]:
# Function to count LSD sentiment terms
def LSDsentiment_count(dict, article):
    """
    Count positive and negative words with negation check. Account for simple negation only for positive words.
    Simple negation is taken to be observations of one of negate words occurring within three words
    preceding a positive words.
    """
    pos_count = 0
    neg_count = 0
 
    pos_words = []
    neg_words = []
 
    input_words=tokenizer(article)    # No lemmatizing since LSD dictionary includes variations
 
    word_count = len(input_words)
 
    for i in range(0, word_count):
        find_neg=pattern_neg_nostar.findall(input_words[i])+pattern_neg_star.findall(input_words[i])
        if len(find_neg)>0:
            if i >= 3:
                if negated(input_words[i - 1]) or negated(input_words[i - 2]) or negated(input_words[i - 3]):
                    pos_count += 1
                    pos_words.append(input_words[i] + ' (with negation)')
                else:
                    neg_count += 1
                    neg_words.append(input_words[i])
            elif i == 2:
                if negated(input_words[i - 1]) or negated(input_words[i - 2]):
                    pos_count += 1
                    pos_words.append(input_words[i] + ' (with negation)')
                else:
                    neg_count += 1
                    neg_words.append(input_words[i])
            elif i == 1:
                if negated(input_words[i - 1]):
                    pos_count += 1
                    pos_words.append(input_words[i] + ' (with negation)')
                else:
                    neg_count += 1
                    neg_words.append(input_words[i])
            elif i == 0:
                neg_count += 1
                neg_words.append(input_words[i])
        
        find_pos=pattern_pos_nostar.findall(input_words[i])+pattern_pos_star.findall(input_words[i])
        if len(find_pos)>0:
            if i >= 3:
                if negated(input_words[i - 1]) or negated(input_words[i - 2]) or negated(input_words[i - 3]):
                    neg_count += 1
                    neg_words.append(input_words[i] + ' (with negation)')
                else:
                    pos_count += 1
                    pos_words.append(input_words[i])
            elif i == 2:
                if negated(input_words[i - 1]) or negated(input_words[i - 2]):
                    neg_count += 1
                    neg_words.append(input_words[i] + ' (with negation)')
                else:
                    pos_count += 1
                    pos_words.append(input_words[i])
            elif i == 1:
                if negated(input_words[i - 1]):
                    neg_count += 1
                    neg_words.append(input_words[i] + ' (with negation)')
                else:
                    pos_count += 1
                    pos_words.append(input_words[i])
            elif i == 0:
                pos_count += 1
                pos_words.append(input_words[i])
    '''
    print('The results with negation check:', end='\n\n')
    print('The # of positive words:', pos_count)
    print('The # of negative words:', neg_count)
    print('The list of found positive words:', pos_words)
    print('The list of found negative words:', neg_words)
    print('\n', end='')
    '''
    
    results = [word_count, pos_count, neg_count, pos_words, neg_words]
 
    return results

In [ ]:
# Run LSD sentiment through all expanded reg sentences
start_time = time.time()

LSDpositiveCount=[]
LSDnegativeCount=[]
LSDpositiveWords=[]
LSDnegativeWords=[]
failed=[]
for i in range(0, len(df['RegSentsExpand'])):
    try:
        results=LSDsentiment_count(LSDdict, df['RegSentsExpand'][i])
    except:
        results=[None, None, None, None, None]
        failed.append(i)        
        
    LSDpositiveCount.append(results[1])
    LSDnegativeCount.append(results[2])
    LSDpositiveWords.append(results[3])
    LSDnegativeWords.append(results[4])
print(len(failed))

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
print(len(failed))
print(len(LSDpositiveWords))

In [ ]:
df['LSDposCount']=LSDpositiveCount
df['LSDnegCount']=LSDnegativeCount
df['LSDposWords']=LSDpositiveWords
df['LSDnegWords']=LSDnegativeWords

In [ ]:
print(df.head())

In [ ]:
for i in range(0,10):
    print(df['RegSentsExpand'][i],df['LSDposWords'][i],df['LSDnegWords'][i])

In [ ]:
df[['ID','LSDposCount','LSDnegCount','LSDposWords','LSDnegWords']].to_csv('/home/ec2-user/SageMaker/New Uncertainty/Sentiment Analysis/LSDsentiments.csv',index=False)